In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasetsforecast.m4 import M4
from config_models import MODEL_LIST

from neuralforecast.losses.numpy import mae, mse, smape, mqloss

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpkpnd172y
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpkpnd172y/_remote_module_non_scriptable.py


## Auto

M4

In [3]:
# Y_df, a, b = M4.load(directory='./', group='Monthly', cache=True)
# y = Y_df.groupby('unique_id').tail(12).reset_index(drop=True)['y']

In [4]:
# source_dataset = 'M4'
# target_dataset = 'M4'
# experiment_id = '20230606'

# models = ['automlp', 'autonhits','autotft','autopatchtst', 'autotcn', 'autolstm']
# model_parse = {
#                 'autonhits': 'NHITS',
#                 'autotft': 'TFT',
#                 'autopatchtst': 'PatchTST',
#                 'automlp': 'MLP',
#                 'autotcn': 'TCN',
#                 'autolstm': 'LSTM',
#               }

# for model in models:
#     model_name = model_parse[model]
#     print('Model:', model_name)
#     results_df = pd.read_csv(f'./results/forecasts/{target_dataset}/{model}_{source_dataset}_{experiment_id}.csv')
#     if target_dataset == 'M4':
#       results_df['y'] = y
#     #results_df = results_df.groupby('unique_id').tail(12)

#     smape_loss = smape(results_df['y'], results_df.iloc[:,3])
#     mae_loss = mae(results_df['y'], results_df.iloc[:,3])
#     print('smape:', smape_loss)
#     print('mae:', mae_loss)

In [5]:
# source_dataset = 'M4'
# target_dataset = 'M3'
# experiment_id = '20230606'

# models = ['automlp', 'autonhits','autotft','autopatchtst', 'autotcn', 'autolstm']
# model_parse = {
#                 'autonhits': 'NHITS',
#                 'autotft': 'TFT',
#                 'autopatchtst': 'PatchTST',
#                 'automlp': 'MLP',
#                 'autotcn': 'TCN',
#                 'autolstm': 'LSTM',
#               }

# for model in models:
#     model_name = model_parse[model]
#     print('Model:', model_name)
#     results_df = pd.read_csv(f'./results/forecasts/{target_dataset}/{model}_{source_dataset}_{experiment_id}.csv')
#     if target_dataset == 'M4':
#       results_df['y'] = y
#     #results_df = results_df.groupby('unique_id').tail(12)

#     smape_loss = smape(results_df['y'], results_df[model_name])
#     mae_loss = mae(results_df['y'], results_df[model_name])
#     print('smape:', smape_loss)
#     print('mae:', mae_loss)

# Fixed

In [39]:
source_dataset = 'M4'
#target_dataset = 'M3'
k_shot=100
experiment_id = '20230621'

#final_df = pd.DataFrame(columns=['model', 'smape', 'mae', 'mqloss'])
mae_df = pd.DataFrame(columns = ['dataset'] + MODEL_LIST)
smape_df = pd.DataFrame(columns = ['dataset'] + MODEL_LIST)
mqloss_df = pd.DataFrame(columns = ['dataset'] + MODEL_LIST)

for target_dataset in ['M3', 'AirPassengers', 'ILI', 'TrafficL']:
    mae_list = []
    smape_list = []
    mqloss_list = []
    for model in MODEL_LIST:
        results_df = pd.read_csv(f'./results/forecasts/{target_dataset}/{model}_{k_shot}_{source_dataset}_{experiment_id}.csv').reset_index(drop=True)
        results_df = results_df.groupby('unique_id').tail(12)

        smape_loss = smape(results_df['y'], results_df.iloc[:,8])
        mae_loss = mae(results_df['y'], results_df.iloc[:,8])
        mq_loss = mqloss(y=results_df['y'], y_hat=results_df.iloc[:,4:13], quantiles=np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))
        mae_list.append(np.round(mae_loss,3))
        smape_list.append(np.round(smape_loss,3))
        mqloss_list.append(np.round(mq_loss,3))

    mae_df = mae_df.append({'dataset': target_dataset, **dict(zip(MODEL_LIST, mae_list))}, ignore_index=True)
    smape_df = smape_df.append({'dataset': target_dataset, **dict(zip(MODEL_LIST, smape_list))}, ignore_index=True)
    mqloss_df = mqloss_df.append({'dataset': target_dataset, **dict(zip(MODEL_LIST, mqloss_list))}, ignore_index=True)

In [40]:
mae_df

,dataset,nhits_15_512,nhits_30_1024,nhits_30_2048,patchtst_128_3,patchtst_512_6,tft_128,tft_512,tft_1024,mlp_512_8,mlp_2048_32,tcn_128_3,tcn_512_5,lstm_128_3,lstm_512_5
0,M3,607.385,610.811,611.100,677.389,642.212,612.204,615.942,613.211,619.173,804.068,665.145,603.483,605.474,615.411
1,AirPassengers,16.038,23.337,25.223,55.007,51.710,13.172,13.917,19.953,29.953,68.688,56.307,55.610,36.014,60.411
2,ILI,0.390,0.387,0.305,0.339,0.307,0.361,0.463,0.463,0.490,0.276,0.609,0.913,0.456,0.853
3,TrafficL,0.858,0.876,0.862,0.907,0.990,0.850,0.802,0.848,0.853,0.385,1.147,1.003,1.062,0.906


In [41]:
smape_df

,dataset,nhits_15_512,nhits_30_1024,nhits_30_2048,patchtst_128_3,patchtst_512_6,tft_128,tft_512,tft_1024,mlp_512_8,mlp_2048_32,tcn_128_3,tcn_512_5,lstm_128_3,lstm_512_5
0,M3,0.137,0.137,0.138,0.147,0.142,0.136,0.138,0.137,0.138,0.170,0.146,0.134,0.134,0.136
1,AirPassengers,0.032,0.048,0.051,0.114,0.106,0.030,0.029,0.045,0.061,0.140,0.122,0.124,0.076,0.133
2,ILI,0.817,0.784,0.742,0.771,0.695,0.962,0.862,0.776,0.962,0.655,0.757,1.294,0.686,0.822
3,TrafficL,1.326,1.344,1.341,1.412,1.404,1.302,1.254,1.298,1.293,0.532,1.647,1.381,1.504,1.328


In [42]:
mqloss_df

,dataset,nhits_15_512,nhits_30_1024,nhits_30_2048,patchtst_128_3,patchtst_512_6,tft_128,tft_512,tft_1024,mlp_512_8,mlp_2048_32,tcn_128_3,tcn_512_5,lstm_128_3,lstm_512_5
0,M3,245.378,246.215,245.903,268.266,259.064,247.651,250.455,248.026,248.209,319.990,269.934,245.784,244.292,248.750
1,AirPassengers,8.899,11.046,13.973,21.095,19.099,5.097,5.801,6.709,13.690,25.207,26.146,22.769,17.129,23.683
2,ILI,0.166,0.152,0.133,0.160,0.174,0.168,0.174,0.181,0.195,0.157,0.235,0.319,0.180,0.333
3,TrafficL,0.389,0.390,0.386,0.334,0.402,0.368,0.361,0.385,0.385,0.174,0.496,0.421,0.456,0.385


In [ ]:
# for i in range(1):
#     unique_id = results_df['unique_id'].unique()[np.random.randint(1)]
#     plot_df = results_df.query('unique_id == @unique_id')

#     plt.figure(figsize=(20, 5))
#     plt.plot(plot_df['y'], label='y')
#     plt.plot(plot_df[model_name], label='y_hat')
#     plt.show()

In [ ]:
import pandas as pd
import numpy as npx

from datasetsforecast.m4 import M4
from datasetsforecast.m3 import M3
from datasetsforecast.long_horizon import LongHorizon
from datasetsforecast.losses import mae
from statsforecast.utils import AirPassengersDF

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, AutoETS, Naive

In [ ]:
target_dataset = 'TrafficL'
horizon = 18

In [ ]:
if (target_dataset == 'AirPassengers'):
    Y_df_target = AirPassengersDF.copy()
    Y_df_target['ds'] = pd.to_datetime(Y_df_target['ds'])
    test_size = horizon
    frequency = 'M'
elif (target_dataset == 'M3'):
    Y_df_target, *_ = M3.load(directory='./', group='Monthly')
    Y_df_target['ds'] = pd.to_datetime(Y_df_target['ds'])
    frequency = 'M'
    test_size = horizon
elif (target_dataset == 'M4'):
    Y_df_target, *_ = M4.load(directory='./', group='Monthly', cache=True)
    Y_df_target['ds'] = pd.to_datetime(Y_df_target['ds'])
    frequency = 'M'
    test_size = horizon
elif (target_dataset == 'ILI'):
    Y_df_target, _, _ = LongHorizon.load(directory='./', group='ILI')
    Y_df_target['ds'] = np.repeat(np.array(range(len(Y_df_target)//7)), 7)
    test_size = horizon
    frequency = 'W'
elif (target_dataset == 'TrafficL'):
    Y_df_target, _, _ = LongHorizon.load(directory='./', group='TrafficL')
    Y_df_target['ds'] = np.repeat(np.array(range(len(Y_df_target)//862)), 862)
    test_size = horizon

In [ ]:
unique_ids = Y_df_target['unique_id'].unique()
unique_ids = np.random.choice(unique_ids,size=20)
Y_df_target = Y_df_target[Y_df_target['unique_id'].isin(unique_ids)].reset_index(drop=True)
Y_df_target = Y_df_target.groupby('unique_id').tail(200).reset_index(drop=True)

In [ ]:
Y_df_target

In [ ]:
season_length = 12 # Monthly data 
#season_length = 24 # Monthly data 

# Include the models you imported
models = [
    AutoARIMA(season_length=season_length),
    #AutoETS(season_length=season_length),
    Naive()
]

# Instansiate the StatsForecast class as sf
sf = StatsForecast(
    df=Y_df_target,
    models=models,
    freq='M', 
    n_jobs=-1
)

crossvalidation_df = sf.cross_validation(
    df = Y_df_target,
    h = horizon,
    step_size = 1,
    n_windows = 1
  )

In [ ]:
crossvalidation_df = crossvalidation_df.reset_index(drop=True)
crossvalidation_df

In [ ]:
for model_name in ['AutoARIMA', 'Naive']:
    print('Model:', model_name)
    mae_loss = mae(crossvalidation_df['y'], crossvalidation_df[model_name])
    print('mae:', mae_loss)